In [0]:
import os
import pyspark
df=spark.createDataFrame(
[
  [0, '카펜터는 130㎞대 후반에서 140㎞대 초반의 패스트볼을 던진다. 공이 빠른 유형은 아니지만 적극적으로 스트라이크존을 공략한다. 100구 이상을 무리 없이 던질 수 있는 체력도 갖췄다.'],

  [0, '김현수는 첫 선발 등판에서 5이닝동안 삼진을 7개나 잡는 무실점 호투로 데뷔 첫 승의 감격을 맛봤습니다.'],

  [0, 'LG는 17일 잠실구장에서 벌어진 롯데 자이언츠와의 경기에서 외국인 에이스 타일러 윌슨의 호투와 이천웅, 김현수의 홈런을 앞세워 9-1로 승리했다.'],

  [0, '2회말 박용택, 이천웅의 연속 안타와 양석환의 희생번트로 1사 2, 3루를 만든 LG는 유강남의 2루 땅볼 때 3루 주자 박용택이 홈인, 선취점을 올렸다.'],

  [0, '이어진 2사 2루에서 박용택이 우중간 적시타를 뽑아내 1점을 보탠 LG는 후속타자 이천웅이 오른쪽 담장을 넘기는 투런 홈런(시즌 3호)을 쏘아올려 5-0으로 점수차를 벌렸다.'],
  [1, '인삼공사는 블로킹과 상대범실로 20-20 동점을 만들었지만 마지막엔 루소를 막지 못했다. 현대건설은 1세트 분위기를 2세트에서도 이어갔다. 고예림과 정지윤의 공격까지 터지면서 승리했다.'],

  [1, '하지만 현대건설의 블로킹이 연이어 터지면서 마침내 추격에 나섰다. 교체투입된 황민경이 정확한 서브로 상대를 흔들면서 16-15 역전까지 성공했다. 현대건설은 안정적으로 경기를 3세트로 마무리했다.'],

  [1, '김연경은 특히 상대의 목적타 서브에 시달리면서도 흔들리지 않았다. 공격은 후배 이재영(19점)에게 양보했다. 김연경은 이날 7점을 올렸다.'],

  [1, '경기 초반 높이로 상대 공격을 막겠다는 전략. 실제로 현대건설 공격수들은 흥국생명의 블로킹 벽을 뚫는 데 어려움을 느꼈다.' ],

  [1, '이재영에게 집중적인 목적타 서브를 구사한 동시에 김연경과 이재영이 뛰어오를 때마다 블로커들을 2∼3명 붙였습니다.']

  ],
  ['cls','sent'])

In [0]:
from pyspark.ml.feature import StringIndexer
labelIndexer=StringIndexer(inputCol="cls",outputCol="label")

In [0]:
from pyspark.ml.feature import RegexTokenizer

regexTok = RegexTokenizer(inputCol="sent", outputCol="wordsRegex", pattern="\\s+")
#tokenizer = Tokenizer(inputCol="sent", outputCol="words")

In [0]:
from pyspark.ml.feature import StopWordsRemover

stop = StopWordsRemover(inputCol="wordsRegex", outputCol="nostops")

In [0]:
stopwords=list()
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)

In [0]:
_mystopwords=[u"나",u"너", u"우리", "take"]
for e in _mystopwords:
    stopwords.append(e)

In [0]:
stop.setStopWords(stopwords)

Out[12]: StopWordsRemover_ef910a1e8389

In [0]:
from pyspark.ml.feature import HashingTF

hashingTF = HashingTF(inputCol="nostops", outputCol="features")

In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[labelIndexer,regexTok,stop,hashingTF])

In [0]:

model=pipeline.fit(df)
trainDf = model.transform(df)

In [0]:
trainDf.select("label","nostops","features").show(truncate=False)

+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
label|nostops |features |
+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
0.0 |[카펜터는, 130㎞대, 후반에서, 140㎞대, 초반의, 패스트볼을, 던진다., 공이, 빠른, 유형은, 아니지만, 적극적으로, 스트라이크존을, 공략한다., 100구, 이상을, 무리, 없이, 던질, 수, 있는, 체력도, 갖췄다.] |(262144,[7829,12282,21456,40362,49571,52729,54077,84240,128886,135308,141739,159669,171007,174781,193229,222387,228643,234445,238127,241878,249904,250998,254325],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
0.0 |[김현수는, 첫, 선발, 등판에서, 5이닝동안, 삼진을, 7개나, 잡는, 무실점, 호투로, 데뷔, 첫, 승의, 감격을, 맛봤습니다.] |(262144,[8496,19325,27020,39656,47440,80738,106698,111752,111842,151386,154348,170580,191267,211129],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0]) |
0.0 |[lg는, 17일, 잠실구장에서, 벌어진, 롯데, 자이언츠와의, 경기에서, 외국인, 에이스, 타일러, 윌슨의, 호투와, 이천웅,, 김현수의, 홈런을, 앞세워, 9-1로, 승리했다.] |(262144,[4464,6634,20671,41759,43116,45575,53336,70755,76995,83755,99444,134447,135932,184472,190711,200074,248285,253547],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
0.0 |[2회말, 박용택,, 이천웅의, 연속, 안타와, 양석환의, 희생번트로, 1사, 2,, 3루를, 만든, lg는, 유강남의, 2루, 땅볼, 때, 3루, 주자, 박용택이, 홈인,, 선취점을, 올렸다.] |(262144,[2410,7459,21419,21700,21907,36284,41759,46908,66332,75716,78506,99111,150760,153909,172228,188807,199462,222488,237352,252823,258325,260168],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
0.0 |[이어진, 2사, 2루에서, 박용택이, 우중간, 적시타를, 뽑아내, 1점을, 보탠, lg는, 후속타자, 이천웅이, 오른쪽, 담장을, 넘기는, 투런, 홈런(시즌, 3호)을, 쏘아올려, 5-0으로, 점수차를, 벌렸다.] |(262144,[20687,21700,22948,31363,39048,41759,52090,58685,62512,79046,81085,86843,92678,111426,120489,134565,139881,190166,193643,199262,238650,244907],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
1.0 |[인삼공사는, 블로킹과, 상대범실로, 20-20, 동점을, 만들었지만, 마지막엔, 루소를, 막지, 못했다., 현대건설은, 1세트, 분위기를, 2세트에서도, 이어갔다., 고예림과, 정지윤의, 공격까지, 터지면서, 승리했다.] |(262144,[23205,24286,27949,60717,70755,98888,109799,112435,120634,123871,135705,136529,152508,152915,164379,170677,171449,181732,228736,252978],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
1.0 |[하지만, 현대건설의, 블로킹이, 연이어, 터지면서, 마침내, 추격에, 나섰다., 교체투입된, 황민경이, 정확한, 서브로, 상대를, 흔들면서, 16-15, 역전까지, 성공했다., 현대건설은, 안정적으로, 경기를, 3세트로, 마무리했다.]|(262144,[10987,15357,15618,21439,50008,68756,103531,113285,136529,145466,162522,164379,165983,169396,183997,191423,194161,201946,212024,230210,244431,249142],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
1.0 |[김연경은, 특히, 상대의, 목적타, 서브에, 시달리면서도, 흔들리지, 않았다., 공격은, 후배, 이재영(19점)에게, 양보했다., 김연경은, 이날, 7점을, 올렸다.] |(262144,[65981,70452,78788,121802,126030,131346,154183,161395,165074,165450,166741,174056,249402,257880,258325],[1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
1.0 |[경기, 초반, 높이로, 상대, 공격을, 막겠다는, 전략., 실제로, 현대건설, 공격수들은, 흥국생명의, 블로킹, 벽을, 뚫는, 데, 어려움을, 느꼈다.] |(262144,[5560,17475,24491,53139,59036,75524,101009,133061,154359,167906,184627,213707,219637,220716,224970,234080,242994],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]) |
1.0 |[이재영에게, 집중적인, 목적타, 서브를, 구사한

In [0]:
trainDf.printSchema()

root
-- cls: long (nullable = true)
-- sent: string (nullable = true)
-- label: double (nullable = false)
-- wordsRegex: array (nullable = true)
 |-- element: string (containsNull = true)
-- nostops: array (nullable = true)
 |-- element: string (containsNull = true)
-- features: vector (nullable = true)

In [0]:
trainDf.select('cls','label','features').show()

+---+-----+--------------------+
cls|label| features|
+---+-----+--------------------+
 0| 0.0|(262144,[7829,122...|
 0| 0.0|(262144,[8496,193...|
 0| 0.0|(262144,[4464,663...|
 0| 0.0|(262144,[2410,745...|
 0| 0.0|(262144,[20687,21...|
 1| 1.0|(262144,[23205,24...|
 1| 1.0|(262144,[10987,15...|
 1| 1.0|(262144,[65981,70...|
 1| 1.0|(262144,[5560,174...|
 1| 1.0|(262144,[9067,200...|
+---+-----+--------------------+

In [0]:
from pyspark.ml.classification import NaiveBayes
nb=NaiveBayes(featuresCol='features', labelCol='label', modelType='multinomial', predictionCol='prediction')

In [0]:
model = nb.fit(trainDf)

In [0]:
model.pi

Out[21]: DenseVector([-0.6931, -0.6931])

In [0]:
model.theta

Out[22]: DenseMatrix(2, 262144, [-12.477, -12.477, -12.477, -12.477, -12.477, -12.477, -12.477, -12.477, ..., -12.477, -12.477, -12.477, -12.477, -12.477, -12.477, -12.477, -12.477], 1)

In [0]:
predictions=model.transform(trainDf)

In [0]:
predictions.select('label','sent','prediction').show()

+-----+----------------------------------+----------+
label| sent|prediction|
+-----+----------------------------------+----------+
 0.0| 카펜터는 130㎞대 후반에서 1...| 0.0|
 0.0| 김현수는 첫 선발 등판에서 5이...| 0.0|
 0.0| LG는 17일 잠실구장에서 벌어...| 0.0|
 0.0| 2회말 박용택, 이천웅의 연속 ...| 0.0|
 0.0| 이어진 2사 2루에서 박용택이 ...| 0.0|
 1.0|인삼공사는 블로킹과 상대범실로 ...| 1.0|
 1.0|하지만 현대건설의 블로킹이 연이...| 1.0|
 1.0| 김연경은 특히 상대의 목적타 서...| 1.0|
 1.0| 경기 초반 높이로 상대 공격을 ...| 1.0|
 1.0|이재영에게 집중적인 목적타 서브...| 1.0|
+-----+----------------------------------+----------+

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

#evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="label", metricName="areaUnderROC")
evaluator=BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="label")

In [0]:
evaluator.evaluate(predictions)

Out[26]: 1.0